In [27]:
import yfinance as yf
import pandas as pd
from sqlalchemy import create_engine
import urllib
from datetime import datetime
from datetime import date
import datetime

In [3]:
quoted = urllib.parse.quote_plus("Driver={SQL Server Native Client 11.0};Server=VIKTOR-PC-2001;Database=DividendPolicyResearch;Trusted_Connection=yes;")
engine = create_engine('mssql+pyodbc:///?odbc_connect={}'.format(quoted))

In [4]:
result = engine.execute('SELECT [Ticker] FROM [dbo].[tb_sp100_tickers]')
df_SP100_companies = pd.DataFrame(result.fetchall())
df_SP100_companies.head()

,0
0,AAPL
1,ABBV
2,ABT
3,ACN
4,ADBE


In [5]:
ticker_list = df_SP100_companies[df_SP100_companies.columns[0]].values.tolist()
len(ticker_list)

101

In [7]:
ticker_list[0:5]

['AAPL', 'ABBV', 'ABT', 'ACN', 'ADBE']

In [19]:
df_splits_final = pd.DataFrame()
df_splits = pd.DataFrame()
i = 1

for ticker in ticker_list:
  stock = yf.Ticker(ticker)

  splits = stock.splits
  df_splits =  pd.DataFrame(splits.to_frame().reset_index())
  df_splits["Ticker"] = ticker

  df_splits_final=pd.concat([df_splits_final, df_splits], ignore_index=True)

  print(str(i) + '. '+ ticker + '. Stock data (rows): ' + str(df_splits.shape[0]) + '.')
  i += 1

1. AAPL. Stock data (rows): 5.
2. ABBV. Stock data (rows): 0.
3. ABT. Stock data (rows): 7.
4. ACN. Stock data (rows): 0.
5. ADBE. Stock data (rows): 6.
6. AIG. Stock data (rows): 13.
7. AMD. Stock data (rows): 4.
8. AMGN. Stock data (rows): 5.
9. AMT. Stock data (rows): 0.
10. AMZN. Stock data (rows): 4.
11. AVGO. Stock data (rows): 0.
12. AXP. Stock data (rows): 6.
13. BA. Stock data (rows): 8.
14. BAC. Stock data (rows): 3.
15. BK. Stock data (rows): 6.
16. BKNG. Stock data (rows): 1.
17. BLK. Stock data (rows): 0.
18. BMY. Stock data (rows): 6.
19. BRK-B. Stock data (rows): 1.
20. C. Stock data (rows): 10.
21. CAT. Stock data (rows): 5.
22. CHTR. Stock data (rows): 0.
23. CL. Stock data (rows): 5.
24. CMCSA. Stock data (rows): 12.
25. COF. Stock data (rows): 1.
26. COP. Stock data (rows): 3.
27. COST. Stock data (rows): 2.
28. CRM. Stock data (rows): 1.
29. CSCO. Stock data (rows): 9.
30. CVS. Stock data (rows): 4.
31. CVX. Stock data (rows): 5.
32. DE. Stock data (rows): 4.
33. DH

In [20]:
df_splits_final

,Date,Stock Splits,Ticker
0,1987-06-16 00:00:00-04:00,2.0,AAPL
1,2000-06-21 00:00:00-04:00,2.0,AAPL
2,2005-02-28 00:00:00-05:00,2.0,AAPL
3,2014-06-09 00:00:00-04:00,7.0,AAPL
4,2020-08-31 00:00:00-04:00,4.0,AAPL
...,...,...,...
448,1976-07-26 00:00:00-04:00,2.0,XOM
449,1981-06-12 00:00:00-04:00,2.0,XOM
450,1987-09-15 00:00:00-04:00,2.0,XOM
451,1997-04-14 00:00:00-04:00,2.0,XOM


In [22]:
df_splits_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 453 entries, 0 to 452
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype                           
---  ------        --------------  -----                           
 0   Date          453 non-null    datetime64[ns, America/New_York]
 1   Stock Splits  453 non-null    float64                         
 2   Ticker        453 non-null    object                          
dtypes: datetime64[ns, America/New_York](1), float64(1), object(1)
memory usage: 10.7+ KB


In [28]:
df_splits_final["date_insert"] = datetime.datetime.now().strftime("%Y-%m-%d")
df_splits_final["time_insert"] = datetime.datetime.now().strftime("%H:%M:%S")
df_splits_final

,Date,Stock Splits,Ticker,date_insert,time_insert
0,1987-06-16 00:00:00-04:00,2.0,AAPL,2024-01-08,23:14:03
1,2000-06-21 00:00:00-04:00,2.0,AAPL,2024-01-08,23:14:03
2,2005-02-28 00:00:00-05:00,2.0,AAPL,2024-01-08,23:14:03
3,2014-06-09 00:00:00-04:00,7.0,AAPL,2024-01-08,23:14:03
4,2020-08-31 00:00:00-04:00,4.0,AAPL,2024-01-08,23:14:03
...,...,...,...,...,...
448,1976-07-26 00:00:00-04:00,2.0,XOM,2024-01-08,23:14:03
449,1981-06-12 00:00:00-04:00,2.0,XOM,2024-01-08,23:14:03
450,1987-09-15 00:00:00-04:00,2.0,XOM,2024-01-08,23:14:03
451,1997-04-14 00:00:00-04:00,2.0,XOM,2024-01-08,23:14:03


In [31]:
result = engine.execute('SELECT COUNT(*) FROM [dbo].[tb_sp100_splits]')
result.fetchall()[0][0]

0

In [32]:
df_splits_final.to_sql('tb_sp100_splits', schema='dbo', con = engine, if_exists='append',index=False)

In [33]:
result = engine.execute('SELECT COUNT(*) FROM [dbo].[tb_sp100_splits]')
result.fetchall()[0][0] 

453